In [4]:
import pandas as pd
import funzioni as fx
from tensorflow.keras.models import load_model
from tensorflow_addons.metrics import F1Score
import yfinance as yf
from multiprocessing import Pool, cpu_count, Manager, Value

def scarica(nome_simbolo):
    try:
        ticker = yf.download(nome_simbolo, start='2021-01-01', end='2023-12-31', progress=False)
        if ticker["Close"].iloc[-1] >= 1:
            ticker.index = ticker.index.date
            ticker = fx.crea_indicatori(ticker)
            ticker = fx.imposta_target(ticker)
            ticker.dropna(axis=0, inplace=True)
            _, X, Y, _ = fx.to_XY(ticker, fx.features_prezzo, fx.features_da_scalare_singolarmente, fx.features_meno_piu, fx.features_candele, fx.features_no_scala, fx.elenco_targets, fx.n_timesteps, fx.giorni_previsione, 0)
            pred = model.predict(X)
            screener.append([nome_simbolo, pred[-1, 0]])  
        return nome_simbolo, X, Y, ""
    except Exception as e:
        return nome_simbolo, None, None, str(e)

def callback_result(result, totale_processati, tot_tickers):
    nome_simbolo, X, Y, err = result
    if X is not None and Y is not None:
        print(f'X.shape:{X.shape}')
        print(f'Y.shape:{Y.shape}')
        print("\033[42m" + f"{totale_processati.value}/{tot_tickers}) Completato ticker {nome_simbolo}" + "\033[0m")
    else:
        print(err)

    with totale_processati.get_lock(): 
        totale_processati.value += 1

f1_score = F1Score(num_classes=1, average='macro', threshold=0.5)
model = load_model("model.h5", custom_objects={'Addons>F1Score': f1_score})

lista_tickers = pd.read_parquet("lista_ticker.parquet")
tot_tickers = len(lista_tickers)

manager = Manager()
screener = manager.list()

totale_processati = Value('i', 0)  

with Pool(cpu_count()) as p:
    for i in range(0, tot_tickers):
        nome_simbolo = lista_tickers.iloc[i]["Ticker"]
        task = (nome_simbolo,)
        p.apply_async(scarica, args=task, callback=lambda result: callback_result(result, totale_processati, tot_tickers))

    p.close()
    p.join()

print("Salvataggio file")
screener_df = pd.DataFrame(list(screener), columns=["Ticker", "Prediction"])
screener_df.to_excel("screener.xlsx")

print("fine")

X.shape:(379, 120, 10)
Y.shape:(379, 1)
12/12 [==============================] - 1s 7ms/step
0/2581) Completato ticker ZIP
X.shape:(478, 120, 10)
Y.shape:(478, 1)
15/15 [==============================] - 0s 7ms/step
1/2581) Completato ticker CE
X.shape:(478, 120, 10)
Y.shape:(478, 1)
15/15 [==============================] - 0s 6ms/step
2/2581) Completato ticker CHEF
X.shape:(478, 120, 10)
Y.shape:(478, 1)
15/15 [==============================] - 0s 6ms/step
3/2581) Completato ticker OOMA
X.shape:(478, 120, 10)
Y.shape:(478, 1)
15/15 [==============================] - 0s 5ms/step
4/2581) Completato ticker PPBI
X.shape:(478, 120, 10)
Y.shape:(478, 1)
15/15 [==============================] - 0s 5ms/step
5/2581) Completato ticker YPF
X.shape:(478, 120, 10)
Y.shape:(478, 1)
15/15 [==============================] - 0s 4ms/step
6/2581) Completato ticker CARR
X.shape:(478, 120, 10)
Y.shape:(478, 1)
15/15 [==============================] - 0s 6ms/step
7/2581) Completato ticker PII
X.shape:(478,


1 Failed download:
['INW']: Exception('%ticker%: No price data found, symbol may be delisted (1d 2021-01-01 -> 2023-12-31)')


15/15 [==============================] - 0s 6ms/step
56/2581) Completato ticker IQV
X.shape:(478, 120, 10)
Y.shape:(478, 1)
15/15 [==============================] - 0s 5ms/step
57/2581) Completato ticker REXR
X.shape:(478, 120, 10)
Y.shape:(478, 1)
15/15 [==============================] - 0s 5ms/step
58/2581) Completato ticker BERY
X.shape:(478, 120, 10)
Y.shape:(478, 1)
15/15 [==============================] - 0s 5ms/step
59/2581) Completato ticker WABC
X.shape:(478, 120, 10)
Y.shape:(478, 1)
15/15 [==============================] - 0s 6ms/step
60/2581) Completato ticker BAK
single positional indexer is out-of-bounds
X.shape:(478, 120, 10)
Y.shape:(478, 1)
15/15 [==============================] - 0s 6ms/step
62/2581) Completato ticker OHI
X.shape:(478, 120, 10)
Y.shape:(478, 1)
15/15 [==============================] - 0s 6ms/step
63/2581) Completato ticker IIIN
X.shape:(478, 120, 10)
Y.shape:(478, 1)
15/15 [==============================] - 0s 7ms/step
64/2581) Completato ticker BANC



1 Failed download:
['MONC']: Exception('%ticker%: No timezone found, symbol may be delisted')


15/15 [==============================] - 0s 5ms/step
74/2581) Completato ticker UTZ
X.shape:(478, 120, 10)
Y.shape:(478, 1)
15/15 [==============================] - 0s 4ms/step
75/2581) Completato ticker SPGI
X.shape:(478, 120, 10)
Y.shape:(478, 1)
15/15 [==============================] - 0s 5ms/step
76/2581) Completato ticker RGLD
X.shape:(478, 120, 10)
Y.shape:(478, 1)
15/15 [==============================] - 0s 5ms/step
77/2581) Completato ticker ABBV
X.shape:(478, 120, 10)
Y.shape:(478, 1)
15/15 [==============================] - 0s 6ms/step
78/2581) Completato ticker GGG
single positional indexer is out-of-bounds
X.shape:(478, 120, 10)
Y.shape:(478, 1)
15/15 [==============================] - 0s 6ms/step
80/2581) Completato ticker SCVL
X.shape:(478, 120, 10)
Y.shape:(478, 1)
15/15 [==============================] - 0s 5ms/step
81/2581) Completato ticker CMG
X.shape:(478, 120, 10)
Y.shape:(478, 1)
15/15 [==============================] - 0s 5ms/step
82/2581) Completato ticker PK
X.

...

15/15 [==============================] - 0s 5ms/step
2455/2581) Completato ticker AMCR
X.shape:(478, 120, 10)
Y.shape:(478, 1)
15/15 [==============================] - 0s 4ms/step
2456/2581) Completato ticker QSR
X.shape:(421, 120, 10)
Y.shape:(421, 1)
14/14 [==============================] - 0s 5ms/step
2457/2581) Completato ticker VZIO
X.shape:(478, 120, 10)
Y.shape:(478, 1)
15/15 [==============================] - 0s 7ms/step
2458/2581) Completato ticker FE
X.shape:(478, 120, 10)
Y.shape:(478, 1)
15/15 [==============================] - 0s 6ms/step
2459/2581) Completato ticker DAKT
X.shape:(478, 120, 10)
Y.shape:(478, 1)
15/15 [==============================] - 0s 10ms/step
2460/2581) Completato ticker DAR
X.shape:(478, 120, 10)
Y.shape:(478, 1)
15/15 [==============================] - 0s 6ms/step
2461/2581) Completato ticker VSEC
single positional indexer is out-of-bounds
X.shape:(478, 120, 10)
Y.shape:(478, 1)
15/15 [==============================] - 0s 5ms/step
2463/2581) Complet


1 Failed download:
['ATCO^PRH']: Exception('%ticker%: No timezone found, symbol may be delisted')


15/15 [==============================] - 0s 4ms/step
2483/2581) Completato ticker RPTX
X.shape:(478, 120, 10)
Y.shape:(478, 1)
15/15 [==============================] - 0s 6ms/step
2484/2581) Completato ticker BDX
X.shape:(478, 120, 10)
Y.shape:(478, 1)
15/15 [==============================] - 0s 5ms/step
2485/2581) Completato ticker EQH
X.shape:(477, 120, 10)
Y.shape:(477, 1)
15/15 [==============================] - 0s 6ms/step
2486/2581) Completato ticker ATRI
X.shape:(478, 120, 10)
Y.shape:(478, 1)
15/15 [==============================] - 0s 6ms/step
2487/2581) Completato ticker UCTT
single positional indexer is out-of-bounds
X.shape:(478, 120, 10)
Y.shape:(478, 1)
 1/15 [=>............................] - ETA: 0s


1 Failed download:
['LDO']: Exception('%ticker%: No price data found, symbol may be delisted (1d 2021-01-01 -> 2023-12-31)')


15/15 [==============================] - 0s 5ms/step
2489/2581) Completato ticker FBP
X.shape:(478, 120, 10)
Y.shape:(478, 1)
15/15 [==============================] - 0s 5ms/step
2490/2581) Completato ticker WIT
X.shape:(478, 120, 10)
Y.shape:(478, 1)
15/15 [==============================] - 0s 6ms/step
2491/2581) Completato ticker FMAO
X.shape:(478, 120, 10)
Y.shape:(478, 1)
15/15 [==============================] - 0s 8ms/step
2492/2581) Completato ticker BEPC
single positional indexer is out-of-bounds
X.shape:(478, 120, 10)
Y.shape:(478, 1)
15/15 [==============================] - 0s 5ms/step
2494/2581) Completato ticker ED
X.shape:(478, 120, 10)
Y.shape:(478, 1)
15/15 [==============================] - 0s 5ms/step
2495/2581) Completato ticker BN
X.shape:(478, 120, 10)
Y.shape:(478, 1)
15/15 [==============================] - 0s 7ms/step
2496/2581) Completato ticker EBF
X.shape:(478, 120, 10)
Y.shape:(478, 1)
15/15 [==============================] - 0s 6ms/step
2497/2581) Completato 